In [1]:
import sympy as sp
import tensorflow as tf
import numpy as np
import MLGeometry as mlg
from MLGeometry import bihomoNN as bnn
import cmath

In [2]:
np.random.seed(42)
tf.random.set_seed(42)

In [3]:
z0, z1, z2, z3, z4 = sp.symbols('z0, z1, z2, z3, z4')
Z = [z0,z1,z2,z3,z4]
f = z0**5 + z1**5 + z2**5 + z3**5 + z4**5 + 0.5*z0*z1*z2*z3*z4

In [4]:
load_path = 'training/f0_psi0.5/300_300_300_1'
model = tf.keras.models.load_model(load_path, compile=False)

In [5]:
points_list = []
n_points = 10000

# The Bihomogeneous layer will remove the zero entries automatically.
# So here we add two small numbers on 0s to avoid being removed
# TODO: Just need one extra point
eps1 = 0.001 + 0.001j
eps2 = 0.001 - 0.002j

for k in range(n_points):
    point_z3 = cmath.exp(np.pi*k*2j/n_points)
    point_z4 = (-1-point_z3**5)**(1/5)
    #points_list.append([1+eps1, 0+eps1, 0+eps2, point_z3+eps1, point_z4+eps1])
    points_list.append([1, 0, 0, point_z3, point_z4])
    
points_list.append(np.random.random(5) + np.random.random(5) * 1j)

In [6]:
curve = mlg.hypersurface.Hypersurface(Z, f, points=points_list, norm_coordinate=0, max_grad_coordinate=3)
curve_dataset = mlg.tf_dataset.generate_dataset(curve).batch(curve.n_points)

In [7]:
@tf.function
def cy_metric(points, restriction):
    
    cy_metric = mlg.complex_math.complex_hessian(tf.math.real(model(points)), points)
    cy_metric_restricted = tf.matmul(restriction, tf.matmul(cy_metric, restriction, adjoint_b=True))
    
    return cy_metric_restricted

In [8]:
points, Omega_Omegabar, mass, restriction = next(iter(curve_dataset))

In [9]:
metric = cy_metric(points, restriction)

/home/spencer/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [39]:
tan_vectors = tf.math.subtract(points, tf.gather(points, list(range(-1, n_points))))

In [40]:
points_rs = tf.reshape(tan_vectors, [n_points+1, 1, 5])
points_cy = tf.gather(points_rs, [1,2,3], axis=2)

In [41]:
tf.reduce_sum(tf.matmul(points_cy, tf.matmul(metric, points_cy, adjoint_b=True)))

<tf.Tensor: shape=(), dtype=complex64, numpy=(1.7324973e+19-165362530000j)>

In [15]:
points_cy

<tf.Tensor: shape=(10000, 1, 3), dtype=complex64, numpy=
array([[[1.0000000e-03+0.001j     , 1.0000000e-03-0.002j     ,
         1.0010000e+00+0.001j     ]],

       [[1.0000000e-03+0.001j     , 1.0000000e-03-0.002j     ,
         1.0009998e+00+0.00162832j]],

       [[1.0000000e-03+0.001j     , 1.0000000e-03-0.002j     ,
         1.0009992e+00+0.00225664j]],

       ...,

       [[1.0000000e-03+0.001j     , 1.0000000e-03-0.002j     ,
         1.0009983e+00-0.00088495j]],

       [[1.0000000e-03+0.001j     , 1.0000000e-03-0.002j     ,
         1.0009992e+00-0.00025664j]],

       [[1.0000000e-03+0.001j     , 1.0000000e-03-0.002j     ,
         1.0009998e+00+0.00037168j]]], dtype=complex64)>

In [16]:
tf.reshape(points_cy, [10000,1,3])

<tf.Tensor: shape=(10000, 1, 3), dtype=complex64, numpy=
array([[[1.0000000e-03+0.001j     , 1.0000000e-03-0.002j     ,
         1.0010000e+00+0.001j     ]],

       [[1.0000000e-03+0.001j     , 1.0000000e-03-0.002j     ,
         1.0009998e+00+0.00162832j]],

       [[1.0000000e-03+0.001j     , 1.0000000e-03-0.002j     ,
         1.0009992e+00+0.00225664j]],

       ...,

       [[1.0000000e-03+0.001j     , 1.0000000e-03-0.002j     ,
         1.0009983e+00-0.00088495j]],

       [[1.0000000e-03+0.001j     , 1.0000000e-03-0.002j     ,
         1.0009992e+00-0.00025664j]],

       [[1.0000000e-03+0.001j     , 1.0000000e-03-0.002j     ,
         1.0009998e+00+0.00037168j]]], dtype=complex64)>

In [19]:
@tf.function
def cy_metric_unrestricted(points):
    
    cy_metric = mlg.complex_math.complex_hessian(tf.math.real(model(points)), points)
    
    return cy_metric

In [29]:
metric_unrestricted =  cy_metric_unrestricted(points)
metric_unrestricted = tf.gather(metric_unrestricted, [1,2,3,4], axis = 2)
metric_unrestricted = tf.gather(metric_unrestricted, [1,2,3,4], axis = 1)

In [35]:
points_rs = tf.reshape(points, [n_points, 1, 5])
points_cy = tf.gather(points_rs, [1,2,3,4], axis=2)

In [36]:
tf.reduce_sum(tf.matmul(points_cy, tf.matmul(metric_unrestricted, points_cy, adjoint_b=True)))

<tf.Tensor: shape=(), dtype=complex64, numpy=(26230.387-0.00022805369j)>

In [28]:
metric_unrestricted

<tf.Tensor: shape=(10000, 4, 4), dtype=complex64, numpy=
array([[[ 1.4972243e+00+9.3714334e-09j,  1.0850620e-03-5.2429037e-05j,
          7.7720458e-04+1.3473514e-04j, -1.2219124e-03+3.0092523e-04j],
        [ 1.0850704e-03+5.2419258e-05j,  1.4985728e+00-3.3178367e-09j,
         -9.1739616e-04-1.3233959e-03j,  1.3878790e-04-1.2980489e-03j],
        [ 7.7717681e-04-1.3472559e-04j, -9.1740314e-04+1.3233874e-03j,
          2.3935678e+00+1.3387762e-09j, -1.0579559e+00-7.6798868e-01j],
        [-1.2219211e-03-3.0087959e-04j,  1.3882248e-04+1.2980092e-03j,
         -1.0579575e+00+7.6798862e-01j,  2.2766221e+00-1.0579824e-06j]],

       [[ 1.4989336e+00+9.4878487e-09j,  7.5341202e-04-2.6454846e-04j,
          5.9553853e-04+2.5654014e-04j,  2.3441756e-04+1.6943621e-03j],
        [ 7.5340876e-04+2.6454171e-04j,  1.4999313e+00-4.0745363e-10j,
         -8.3577400e-04-1.0011492e-03j, -1.1746248e-03-7.7784358e-04j],
        [ 5.9553469e-04-2.5654514e-04j, -8.3578995e-04+1.0011698e-03j,
          2.